In [2]:
# Example Notebook file demonstrating how to use the file structure

from utils.preprocess_util_lib_example import save_random_dataframe
from pathlib import Path

save_random_dataframe(Path("../output"), Path("test.csv"))